In [2]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from __future__ import division, print_function, absolute_import
from tf_utils import load_features_with_deltas_stacking, random_mini_batches

train_data, test_data, train_labels, test_labels = load_features_with_deltas_stacking()
train_data = train_data.astype(np.float32)
test_data = test_data.astype(np.float32)

tf.logging.set_verbosity(tf.logging.INFO)

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:962: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:962: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))


In [3]:
#Seeding
seed = 3

# Network Parameters
input_h = train_data.shape[1] #Height
input_w = train_data.shape[2] #Width
num_classes = 2
learning_rate = 0.01

In [10]:
def sound_net(features, labels, mode):

    predictions = None
    loss = None
    train_op = None
    eval_metric_ops = None
    keep_prob = tf.constant(0.9, dtype='float32')    
    
    # Input Layer
    x = tf.reshape(features["x"], [-1, input_h, input_w, 1])

    # Convolutional Layer #1
    # input shape [batch_size, input_h, input_w, 1]
    # output shape [batch_size, input_h, input_w, 80]   
    conv1 = tf.layers.conv2d(inputs=x, filters=80, kernel_size=[57, 6], 
                             padding="same", activation=tf.nn.relu, name='conv1')

    # Pooling Layer #1
    # First max pooling layer with a 2x2 filter and stride of 2
    # input shape [batch_size, input_h, input_w, 80]
    # output shape [batch_size, input_h/2, input_w/2, 80]
    pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], padding="same", strides=2)
    pool1 = tf.layers.dropout(inputs=pool1, rate=keep_prob, training=mode == tf.estimator.ModeKeys.TRAIN, name='pool1')

    # Convolutional Layer #2
    # input shape [batch_size, input_h/2, input_w/2, 80]
    # output shape [batch_size, input_h/2, input_w/2, 80]
    conv2 = tf.layers.conv2d(inputs=pool1, filters=80, kernel_size=[1, 3], 
                             padding="same", activation=tf.nn.relu, name='conv2')
    
    # Pooling Layer #2
    # Input Tensor Shape: [batch_size, input_h/2, input_w/2, 80]
    # Output Tensor Shape: [batch_size, input_h/16 input_w/14, 80]    
    pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[8, 7], padding="same", 
                                    strides=[8, 7], name='pool1')

    # Fully connected layer #1
    # Reshape conv2 output to fit fully connected layer input
    pool2_flat = tf.reshape(pool2, [-1, int(input_h/16)*int(input_w/14)*80])    
    dense1 = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)
    dense1 = tf.layers.dropout(inputs=dense1, rate=keep_prob, training=mode == tf.estimator.ModeKeys.TRAIN)

    # Fully connected layer #2
    dense2 = tf.layers.dense(inputs=dense1, units=1024, activation=tf.nn.relu)
    dense2 = tf.layers.dropout(inputs=dense2, rate=keep_prob, training=mode == tf.estimator.ModeKeys.TRAIN)    
    
    # Logits layer
    logits = tf.layers.dense(inputs=dense2, units=num_classes)

    predictions = {
      "classes": tf.argmax(input=logits, axis=1),
      "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
    }
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

    # Calculate Loss (for both TRAIN and EVAL modes)
    loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=labels), name='loss_tensor')
#     accuracy = tf.metrics.accuracy(labels=labels, predictions=predictions["classes"], name='accuracy_tensor')
#     loss = tf.losses.softmax_cross_entropy(labels=labels, logits=logits)
#     loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)
#     loss_var = loss_var.assign(loss)
#     mse_loss = loss_var.assign(loss)
        
    eval_metric_ops = {
      "accuracy": tf.metrics.accuracy(labels=labels, predictions=predictions["classes"])}
    
    # Configure the Training Op (for TRAIN mode)
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
        train_op = optimizer.minimize(
            loss=loss,
            global_step=tf.train.get_global_step())
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op, eval_metric_ops=eval_metric_ops)

  # Add evaluation metrics (for EVAL mode)
    eval_metric_ops = {
      "accuracy": tf.metrics.accuracy(labels=labels, predictions=predictions["classes"])}
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

In [11]:
def main(unused_argv):

    # Create the Estimator or warm-start the last one
    sound_classifier = tf.estimator.Estimator(
      model_fn=sound_net, model_dir="/tmp/sound_classifier_model_lr_0_01_dr_0_1")
    
    # Set up logging for predictions
    # Log the values in the "Softmax" tensor with label "probabilities"
    tensors_to_log = {"loss": "loss_tensor"}
    logging_hook = tf.train.LoggingTensorHook(tensors_to_log, every_n_iter=100)

    # Train the model
    train_input_fn = tf.estimator.inputs.numpy_input_fn(
      x={"x": train_data},
      y=train_labels,
      batch_size=64,
      num_epochs=None,
      shuffle=True)
    
    sound_classifier.train(
      input_fn=train_input_fn,
      steps=10000,
      hooks=[logging_hook])

    # Evaluate the model and print results
    eval_input_fn = tf.estimator.inputs.numpy_input_fn(
      x={"x": test_data},
      y=test_labels,
      num_epochs=1,
      shuffle=False)
    eval_results = sound_classifier.evaluate(input_fn=eval_input_fn)
    print(eval_results)

In [12]:
if __name__ == "__main__":
    tf.app.run()

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/sound_classifier_model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f3e931b0ac8>, '_task_type': 'worker', '_task_id': 0, '_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Create CheckpointSaverHook.


ResourceExhaustedError: OOM when allocating tensor with shape[64,256,28,80] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[Node: conv2d/Conv2D = Conv2D[T=DT_FLOAT, data_format="NHWC", dilations=[1, 1, 1, 1], padding="SAME", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true, _device="/job:localhost/replica:0/task:0/device:GPU:0"](Reshape, conv2d/kernel/read)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[Node: GradientDescent/update_conv2d/bias/ApplyGradientDescent/_78 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_313_GradientDescent/update_conv2d/bias/ApplyGradientDescent", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]
Hint: If you want to see a list of 

In [1]:
tf.reset_default_graph()

NameError: name 'tf' is not defined